In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os

In [2]:
def Sobel(image):
    scale = 1
    delta = 0
    ddepth = cv2.CV_16S
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    grad_x = cv2.Sobel(gray, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    grad_y = cv2.Sobel(gray, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    return grad,abs_grad_x,abs_grad_y

def image_to_sum_array(image):
    (a,b,c) = Sobel(image)
    high = []
    for line in c:
        high.append(sum(line))
    return np.array(high)

def image_to_sobel(image):
    (a,b,c) = Sobel(image)
    return np.array(c)

In [3]:
model = tf.keras.models.load_model("Model_2.h5")

# Single Image Test

In [4]:
image = cv2.imread(filename="Dataset/A/WIN_20221225_15_08_12_Pro.jpg")
# images_list = []
# images_list.append(np.array(image_to_sobel(image)))
# x1 = np.asarray(images_list)
# images_list = []
# images_list.append(np.array(image_to_sum_array(image)))
# x2 = np.asarray(images_list)
x1 = np.array([np.array(image_to_sobel(image))])
x2 = np.array([np.array(image_to_sum_array(image))])
a = model.predict_on_batch([x1,x2])

In [6]:
np.argmax(a)

0

# Folder Test

In [ ]:
!unzip "Test Dataset.zip" -d "Dataset1"

In [31]:
a_list = os.listdir("drive/MyDrive/Colab Notebooks/Dataset1/A/")
b_list = os.listdir("drive/MyDrive/Colab Notebooks/Dataset1/B/")
c_list = os.listdir("drive/MyDrive/Colab Notebooks/Dataset1/C/")

In [32]:
X1 = np.empty((len(a_list)+len(b_list)+len(c_list),480,640))
X2 = np.empty((len(a_list)+len(b_list)+len(c_list),480))
Y = np.empty((len(a_list)+len(b_list)+len(c_list),3))

In [33]:
for i in range(len(a_list)):
    image = cv2.imread(filename="drive/MyDrive/Colab Notebooks/Dataset1/A/"+a_list[i])
    X1[i] = image_to_sobel(image)
    X2[i] = image_to_sum_array(image)
    Y[i] = [1,0,0]

for i in range(len(b_list)):
    image = cv2.imread(filename="drive/MyDrive/Colab Notebooks/Dataset1/B/"+b_list[i])
    X1[i+len(a_list)] = image_to_sobel(image)
    X2[i+len(a_list)] = image_to_sum_array(image)
    Y[i+len(a_list)] = [0,1,0]

for i in range(len(c_list)):
    image = cv2.imread(filename="drive/MyDrive/Colab Notebooks/Dataset1/C/"+c_list[i])
    X1[i+len(a_list)+len(b_list)] = image_to_sobel(image)
    X2[i+len(a_list)+len(b_list)] = image_to_sum_array(image)
    Y[i+len(a_list)+len(b_list)] = [0,0,1]

In [34]:
y_pred = model.predict([X1,X2],batch_size=8, verbose=1)

5/5 [==============================] - 24s 4s/step


In [35]:
from sklearn.metrics import confusion_matrix

In [37]:
y_prediction = np.argmax (y_pred, axis = 1)
y_test=np.argmax(Y, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_prediction)
print(result)

[[11  0  0]
 [ 0 14  0]
 [ 0  0  8]]


In [38]:
model.evaluate([X1,X2],Y,batch_size=8,verbose=1)

5/5 [==============================] - 20s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]